In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate

In [2]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

# Data creation

In [3]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
    display(pret)

matrices by Pakulina A. N.

In [4]:
a1=np.matrix('-401.52 200.16; 1200.96 -601.68')
a1=np.array(a1)
pretty_view(a1)

,1,2
1,-401.52000000,200.16000000
2,1200.96000000,-601.68000000


In [5]:
a2=np.matrix('-403.15 200.95; 1205.70 -604.10')
a2=np.array(a2)
pretty_view(a2)

,1,2
1,-403.15000000,200.95000000
2,1205.70000000,-604.10000000


Hilbert matrices of order 2, 3

In [6]:
h2=linalg.hilbert(2)
pretty_view(h2)

,1,2
1,1.00000000,0.50000000
2,0.50000000,0.33333333


In [7]:
h3=linalg.hilbert(3)
pretty_view(h3)

,1,2,3
1,1.00000000,0.50000000,0.33333333
2,0.50000000,0.33333333,0.25000000
3,0.33333333,0.25000000,0.20000000


Diagonally dominant matrix

In [8]:
diag_m=np.matrix('''6 -1 0 0 0;
                    -1 6 -1 0 0;
                    0 -1 6 -1 0;
                    0 0 -1 6 -1;
                    0 0 0 -1 6''')
diag_m=np.array(diag_m)
pretty_view(diag_m)

,1,2,3,4,5
1,6,-1,0,0,0
2,-1,6,-1,0,0
3,0,-1,6,-1,0
4,0,0,-1,6,-1
5,0,0,0,-1,6


# Counting coefs and methods realisation

In [9]:
def count_b(A, x):  # считаем b для последующих вычислений
    b = np.dot(A, x)
    return b

In [10]:
def count_coefs(A, b):  # считаем alpha и beta
    alpha = np.array(np.zeros((A.shape[0], A.shape[0])))
    beta = np.array(np.zeros(b.shape[0]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i != j:
                alpha[i][j] = - A[i][j] / A[i][i]
                beta[i] = b[i] / A[i][i]
            else:
                alpha[i][i] = 0
    return alpha, beta

In [11]:
def simple_iteration_method(alpha, beta, x, eps):  # метод простой итерации
    n_iter=1
    err = eps + 1
    while err > eps:
        err = np.linalg.norm(np.dot(alpha, x) + beta - x)
        x = np.dot(alpha, x) + beta
        n_iter += 1
    x = np.dot(alpha, x) + beta
    return x, n_iter

In [12]:
def seidel_method(A, b, eps): # функция возвращает x  и кол-во итераций для сравнения
    n_iter = 0
    x = np.array(np.zeros((b.shape[0])))
    err = eps + 1
    while err > eps:
        x_n = x.copy()
        for i in range(A.shape[0]):
            x1 = sum(A[i][j] * x_n[j] for j in range(i))
            x2 = sum(A[i][j] * x[j] for j in range(i + 1, A.shape[0]))
            x_n[i] = (b[i] - x1 - x2)/A[i][i]
        err = np.linalg.norm(x_n - x)
        n_iter += 1
        x = x_n
    return x, n_iter

# Print results

In [13]:
def print_result(A):
    pretty_view(A)
    x_w = np.random.uniform(0, 100, size=A.shape[0])
    b = count_b(A, x_w)
    alpha, beta = count_coefs(A, b)
    table=[['Приближение', "n_iter м. простых итераций","n_iter м. Зейделя", 
                'x - x_sim', 'x - x_seid']]
    for eps in (1e-4, 1e-7, 1e-10, 1e-13):
        row=[]
        row.append(eps)
        x_sim, n_iter_1 = simple_iteration_method(alpha, beta, beta, eps)
        x_seid, n_iter_2 = seidel_method(A, b, eps)
        row.append(n_iter_1)
        row.append(n_iter_2)
        row.append(np.linalg.norm(x_w - x_sim))
        row.append(np.linalg.norm(x_w- x_seid))
        table.append(row)
    return print(tabulate(table, tablefmt='fancy_grid', numalign="right"))

In [14]:
list_of_matrixes=[a1, a2, h2, diag_m]

In [15]:
for matrix in list_of_matrixes:
    print_result(matrix)

,1,2
1,-401.52000000,200.16000000
2,1200.96000000,-601.68000000


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 5535                       │ 1248              │ 4.981478275381603e-05  │ 0.019898397647805766   │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 8303                       │ 2632              │ 4.992426167448611e-08  │ 1.994326529456602e-05  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 11073                      │ 4016              │ 4.7127963935531873e-11 │ 1.9986171495829062e-08 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

,1,2
1,-403.15000000,200.95000000
2,1205.70000000,-604.10000000


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 4685                       │ 1285              │ 0.00010764782986164293 │ 0.019171438850541504   │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 7353                       │ 2619              │ 1.0786178257484547e-07 │ 1.920958462860987e-05  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 10023                      │ 3953              │ 1.0725390461281745e-10 │ 1.924716222437619e-08  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

,1,2
1,1.00000000,0.50000000
2,0.50000000,0.33333333


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 103                        │ 40                │ 3.2433241997382446e-05 │ 0.00026994082117933783 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 151                        │ 64                │ 3.254323834509274e-08  │ 2.7085628735113524e-07 │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 199                        │ 88                │ 3.267049556694503e-11  │ 2.717691242272817e-10  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

,1,2,3,4,5
1,6,-1,0,0,0
2,-1,6,-1,0,0
3,0,-1,6,-1,0
4,0,0,-1,6,-1
5,0,0,0,-1,6


╒═════════════╤════════════════════════════╤═══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter м. простых итераций │ n_iter м. Зейделя │ x - x_sim              │ x - x_seid             │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 13                         │ 8                 │ 3.6202095750484374e-06 │ 7.110253728164899e-06  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-07       │ 18                         │ 11                │ 7.257390867881125e-09  │ 4.12619155543137e-09   │
├─────────────┼────────────────────────────┼───────────────────┼────────────────────────┼────────────────────────┤
│ 1e-10       │ 24                         │ 14                │ 4.200227205702084e-12  │ 2.392039771862171e-12  │
├─────────────┼────────────────────────────┼───────────────────┼────────────────

# Метод релаксации

In [16]:
# import warnings
# warnings.filterwarnings("ignore")

In [17]:
def relax_method(A, b, x):
    n = A.shape[0]
    x = x.copy()
    mask = np.full(n, True)
    for j in range(n):
        deltas = A @ x - b
        k = np.nanargmax(np.abs(np.where(mask, deltas, np.nan)))
        if A[k, j] != 0:
            x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j] 
        mask[k] = False
    return x

In [18]:
def relax_solve(A, b, x, eps):
    n_iter = 1
    err = eps + 1
    while err > eps:
        x_new = relax_method(A, b, x)
        err = np.linalg.norm(x - x_new)
        n_iter += 1
        x = x_new
    return x, n_iter

In [19]:
def test_data():
    n = 201
    A = np.random.rand(n, n)
    for i in range(n):
        A[i, i] = sum([abs(A[i, j]) for j in range(n)])
    return A

In [20]:
def var(value, eps):
    return np.random.choice([-eps, eps], size=value.shape) + value

In [21]:
def var_():
    return np.random.uniform(0, 201, size=A.shape[0])

In [18]:
A = test_data()
x = np.random.uniform(0, 201, size=A.shape[0])
b = count_b(A, x)
for i in range(50):
    A_2 = var(A, 1e-3)
    for j in range(50):
        try:
            start_x = var_()
            x_2 = relax_solve(A_2, b, start_x, 1)
            print('Решение найдено!')
            print('Норма разности', np.linalg.norm(x_2 - x))
            break
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:
            pass

<ipython-input-8-ebb96acb7a02>:9: RuntimeWarning: overflow encountered in matmul
  x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j]
<ipython-input-8-ebb96acb7a02>:6: RuntimeWarning: invalid value encountered in matmul
  deltas = A @ x - b
<ipython-input-8-ebb96acb7a02>:9: RuntimeWarning: overflow encountered in double_scalars
  x[j] = (b[k] - A[k] @ x + A[k, j]*x[j]) / A[k, j]


Решение найдено!
Норма разности 7.53982713e-07
